In [ ]:
# this is the data exploration file - import CSVs or connecting to SQLite, basic EDA, etc. 

In [185]:

import pandas as pd


df = pd.read_csv("/Users/lukemorrison/Desktop/nba-bucks-margin-by-quarter/notebooks/csv/game_summary.csv")
df_line = pd.read_csv("/Users/lukemorrison/Desktop/nba-bucks-margin-by-quarter/notebooks/csv/line_score.csv")
df.head()

df.columns

df_line.columns

mask_bucks = (
    (df_line["team_nickname_home"] == "Bucks") |
    (df_line["team_nickname_away"] == "Bucks")
)
df_bucks = df_line[mask_bucks].copy()


def is_bucks_home(row):
    return row["team_nickname_home"] == "Bucks"

df_bucks["bucks_home"] = df_bucks.apply(is_bucks_home, axis=1)


def get_bucks_points(row, quarter):
    home_key = f"pts_qtr{quarter}_home"
    away_key = f"pts_qtr{quarter}_away"
    if row["bucks_home"]:
        return row[home_key]
    else:
        return row[away_key]

def get_opp_points(row, quarter):
    home_key = f"pts_qtr{quarter}_home"
    away_key = f"pts_qtr{quarter}_away"
    if row["bucks_home"]:
        return row[away_key]
    else:
        return row[home_key]


for q in [1, 2, 3, 4]:
    bucks_col = f"q{q}_bucks"
    opp_col   = f"q{q}_opp"
    df_bucks[bucks_col] = df_bucks.apply(lambda r: get_bucks_points(r, q), axis=1)
    df_bucks[opp_col]   = df_bucks.apply(lambda r: get_opp_points(r, q), axis=1)


df_bucks["bucks_total_4q"] = (
    df_bucks["q1_bucks"] +
    df_bucks["q2_bucks"] +
    df_bucks["q3_bucks"] +
    df_bucks["q4_bucks"]
)
df_bucks["opp_total_4q"] = (
    df_bucks["q1_opp"] +
    df_bucks["q2_opp"] +
    df_bucks["q3_opp"] +
    df_bucks["q4_opp"]
)


q_summary = []
for q in [1, 2, 3, 4]:
    bucks_col = f"q{q}_bucks"
    opp_col   = f"q{q}_opp"
    avg_bucks = df_bucks[bucks_col].mean()
    avg_opp   = df_bucks[opp_col].mean()
    q_summary.append((q, avg_bucks, avg_opp))

df_quarterly_avg = pd.DataFrame(q_summary, columns=["Quarter", "BucksAvg", "OppAvg"])
print("Quarterly Averages:")
print(df_quarterly_avg)


for q in [1, 2, 3, 4]:
    margin_col = f"q{q}_margin"
    df_bucks[margin_col] = df_bucks[f"q{q}_bucks"] - df_bucks[f"q{q}_opp"]


for q in [1, 2, 3, 4]:
    margin_col = f"q{q}_margin"
    avg_margin = df_bucks[margin_col].mean()
    print(f"Avg margin in Q{q} = {avg_margin:.2f}")


df_quarterly_avg["MarginAvg"] = df_quarterly_avg["BucksAvg"] - df_quarterly_avg["OppAvg"]

df_quarterly_avg.to_csv("bucks_quarterly_scoring.csv", index=False)
print("\nFinal summary saved to 'bucks_quarterly_scoring.csv'")


Quarterly Averages:
   Quarter   BucksAvg     OppAvg
0        1  26.343291  25.897934
1        2  25.796365  25.624347
2        3  25.917351  25.512074
3        4  25.745332  25.678616
Avg margin in Q1 = 0.45
Avg margin in Q2 = 0.17
Avg margin in Q3 = 0.41
Avg margin in Q4 = 0.07

Final summary saved to 'bucks_quarterly_scoring.csv'


Earliest Bucks game_date_est: 1968-10-16 00:00:00
Latest Bucks game_date_est: 2023-04-26 00:00:00
